d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Querying Files with SQL

Apache Spark&trade; and Databricks&reg; allow you to use SQL to query large data files.

## In this lesson you:
* Query large files using Spark SQL
* Visualize query results using charts
* Create temporary views to simplify complex queries 

## Audience
* Primary Audience: Data Analysts
* Additional Audiences: Data Engineers and Data Scientists

## Prerequisites
* Web browser: **Chrome**
* A cluster configured with **8 cores** and **DBR 6.3**
* Familiarity with <a href="https://www.w3schools.com/sql/" target="_blank">ANSI SQL</a> is required

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup & Classroom-Cleanup<br>

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [4]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/glq179t3sr?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/glq179t3sr?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

### Querying Tables
This lesson uses the table `People10m`. 

The data is fictitious; in particular, the Social Security numbers are fake.

<iframe  
src="//fast.wistia.net/embed/iframe/wqp0pe2mol?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/wqp0pe2mol?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [8]:
%sql
CREATE TABLE IF NOT EXISTS People10M
USING parquet
OPTIONS (
  path "/mnt/training/dataframes/people-10m.parquet"
)

In [9]:
%sql
SELECT * FROM People10M order by birthDate limit 10

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
7677553,Marcus,Arlie,Tibb,M,1951-12-31T05:00:00.000+0000,999-90-6698,47746
8206921,Sylvester,Faustino,Glasgow,M,1951-12-31T05:00:00.000+0000,907-30-2443,43279
7736171,Pat,Johnie,De Keep,M,1951-12-31T05:00:00.000+0000,960-92-7355,98868
8050195,Anibal,Nicky,Springford,M,1951-12-31T05:00:00.000+0000,998-90-1376,62784
8169581,Jc,Edison,Fruchter,M,1951-12-31T05:00:00.000+0000,905-35-8052,58120
8053263,Karl,Horace,Thompson,M,1951-12-31T05:00:00.000+0000,930-70-7868,90481
7813417,Brooks,Dannie,Lemmens,M,1951-12-31T05:00:00.000+0000,914-82-1490,69065
8125007,Quinton,Eduardo,Rooze,M,1951-12-31T05:00:00.000+0000,903-21-6089,87389
8037265,Dexter,Otis,Brahmer,M,1951-12-31T05:00:00.000+0000,959-50-1621,91473
7929094,Milan,Alex,Setterfield,M,1951-12-31T05:00:00.000+0000,989-42-1725,79877


In [10]:
%sql
Select count(id) from People10M

count(id)
10000000


Take a look at its schema with the `DESCRIBE` function.

In [12]:
%sql
DESCRIBE People10M

col_name,data_type,comment
id,int,null
firstName,string,null
middleName,string,null
lastName,string,null
gender,string,null
birthDate,timestamp,null
ssn,string,null
salary,int,null


A simple SQL statement can answer the following question:
> According to our data, which women were born after 1990?

In Databricks, a `SELECT` statement in a SQL cell is automatically run through Spark, and the results are displayed in an HTML table.

In [14]:
%sql
SELECT firstName, middleName, lastName, birthDate
FROM People10M
WHERE year(birthDate) > 1990 AND gender = 'F'
LIMIT 10

firstName,middleName,lastName,birthDate
An,Amira,Cowper,1992-02-08T05:00:00.000+0000
Caroyln,Mamie,Cardon,1994-05-15T04:00:00.000+0000
Yesenia,Eileen,Goldring,1997-07-09T04:00:00.000+0000
Hedwig,Dulcie,Pendleberry,1998-12-02T05:00:00.000+0000
Kala,Violeta,Lyfe,1994-06-23T04:00:00.000+0000
Gussie,India,McKeeman,1991-11-15T05:00:00.000+0000
Pansy,Suzie,Shrieves,1991-05-24T04:00:00.000+0000
Chung,Dian,Dautry,1998-01-12T05:00:00.000+0000
Erica,Louvenia,O'Drought,1991-03-08T05:00:00.000+0000
Katelyn,Merrie,Pocklington,1994-01-16T05:00:00.000+0000


### Built-in functions

Spark provides a number of <a href="https://spark.apache.org/docs/latest/api/sql/" target="_blank">built-in functions</a>, many of which can be used directly from SQL.  These functions can be used in the `WHERE` expressions to filter data and in `SELECT` expressions to create derived columns.

The following SQL statement finds women born after 1990; it uses the `year` function, and it creates a `birthYear` column on the fly.

<iframe  
src="//fast.wistia.net/embed/iframe/jsd9u7ep3k?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/jsd9u7ep3k?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [17]:
%sql
SELECT firstName, middleName, lastName, year(birthDate) as birthYear, salary 
FROM People10M
WHERE year(birthDate) > 1990 AND gender = 'F'
LIMIT 10

firstName,middleName,lastName,birthYear,salary
An,Amira,Cowper,1992,40203
Caroyln,Mamie,Cardon,1994,60449
Yesenia,Eileen,Goldring,1997,73060
Hedwig,Dulcie,Pendleberry,1998,60857
Kala,Violeta,Lyfe,1994,101601
Gussie,India,McKeeman,1991,46945
Pansy,Suzie,Shrieves,1991,73811
Chung,Dian,Dautry,1998,47190
Erica,Louvenia,O'Drought,1991,80113
Katelyn,Merrie,Pocklington,1994,77925


<iframe  
src="//fast.wistia.net/embed/iframe/pl68ybkps2?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/pl68ybkps2?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
### Visualization

Databricks provides built-in easy to use visualizations for your data. 

Take the query below, and visualize it by selecting the bar graph icon once the table is displayed:

<img src="https://files.training.databricks.com/images/eLearning/visualization-1.png" style="border: 1px solid #aaa; padding: 10px; border-radius: 10px 10px 10px 10px"/>

How many women were named Mary in seach year?

In [21]:
%sql
SELECT year(birthDate) as birthYear, count(*) AS total
FROM People10M
WHERE firstName = 'Mary' AND gender = 'F'
GROUP BY birthYear
ORDER BY birthYear

birthYear,total
1952,27
1953,25
1954,15
1955,23
1956,28
1957,29
1958,26
1959,28
1960,37
1961,29


Compare popularity of two names from 1990

In [23]:
%sql
SELECT year(birthDate) as birthYear,  firstName, count(*) AS total
FROM People10M
WHERE (firstName = 'Dorothy' or firstName = 'Donna') AND gender = 'F' AND year(birthDate) > 1990
GROUP BY birthYear, firstName
ORDER BY birthYear, firstName

birthYear,firstName,total
1991,Donna,27
1991,Dorothy,25
1992,Donna,30
1992,Dorothy,39
1993,Donna,22
1993,Dorothy,28
1994,Donna,26
1994,Dorothy,21
1995,Donna,33
1995,Dorothy,26


### Temporary Views

Temporary views assign a name to a query that will be reused as if they were tables themselves. Unlike tables, temporary views aren't stored on disk and are visible only to the current user. This course makes use of temporary views in the exercises to enable the test cases to verify your queries are correct.

A temporary view gives you a name to query from SQL, but unlike a table, it exists only for the duration of your Spark Session. As a result, the temporary view will not carry over when you restart the cluster or switch to a new notebook. It also won't show up in the Data tab that, linked on the left of a Databricks notebook, provides easy access to databases and tables.

The following statement creates a temporary view containing the same data.

<iframe  
src="//fast.wistia.net/embed/iframe/kh6opy2t14?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/kh6opy2t14?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [26]:
%sql
CREATE OR REPLACE TEMPORARY VIEW TheDonnas AS
  SELECT * 
  FROM People10M 
  WHERE firstName = 'Donna'

To view the contents of temporary view, use select notation

In [28]:
%sql
SELECT * FROM TheDonnas LIMIT 10

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
2595,Donna,Carola,Philipot,F,1964-09-26T04:00:00.000+0000,999-24-1601,63160
19295,Donna,Dot,Bonnier,F,1954-05-07T04:00:00.000+0000,925-80-5759,73349
22411,Donna,Teri,Prati,F,1987-06-03T04:00:00.000+0000,993-90-1475,64459
23875,Donna,Elene,August,F,1993-01-06T05:00:00.000+0000,985-11-2169,37634
25491,Donna,Goldie,Cootes,F,1977-08-24T04:00:00.000+0000,963-80-5780,91655
28788,Donna,Shaunte,Perch,F,1971-02-10T05:00:00.000+0000,988-80-1426,88400
29051,Donna,Jeannette,Barthropp,F,1984-09-13T04:00:00.000+0000,915-92-2763,59860
31223,Donna,Esther,Pucker,F,1985-06-19T04:00:00.000+0000,996-69-9290,99457
33314,Donna,Lura,Bento,F,1956-05-12T04:00:00.000+0000,920-63-1598,62760
39507,Donna,Particia,Illing,F,1980-11-04T05:00:00.000+0000,923-86-4682,98838


Create more complex query from People10M table

In [30]:
%sql
CREATE OR REPLACE TEMPORARY VIEW WomenBornAfter1990 AS
  SELECT firstName, middleName, lastName, year(birthDate) AS birthYear, salary 
  FROM People10M
  WHERE year(birthDate) > 1990 AND gender = 'F'

Once a temporary view has been created, it can be queried as if it were itself a table. Find out how many Marys are in the WomenBornAfter1990 view.

In [32]:
%sql
SELECT birthYear, count(*) 
FROM WomenBornAfter1990 
WHERE firstName = 'Mary' 
GROUP BY birthYear 
ORDER BY birthYear

birthYear,count(1)
1991,25
1992,29
1993,39
1994,27
1995,28
1996,35
1997,32
1998,27
1999,25
2000,1


-sandbox
## Exercise 1

Create a temporary view called `Top10FemaleFirstNames` that contains the 10 most common female first names in the `People10M` table. The view must have two columns:

* `firstName` - the first name
* `total` - the total number of rows with that first name

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** You may need to break ties by firstName because some of the totals are identical

Display the results.

### Step 1
Create the temporary view.

In [35]:
%sql
create or replace temporary view Top10FemaleFirstNames as 
  select firstName, count(firstName) as total 
  from People10M
  WHERE gender = 'F'
  group by firstName
  ORDER BY count(firstName) desc
  limit 10
  

In [36]:
%sql
SELECT * from Top10FemaleFirstNames ORDER BY firstName


firstName,total
Alesha,1368
Alice,1384
Bridgette,1373
Cristen,1375
Jacquelyn,1381
Katherin,1373
Lashell,1387
Louie,1382
Lucille,1384
Sharyn,1394


In [37]:
# TEST - Run this cell to test your solution.

resultsDF = spark.sql("SELECT * FROM Top10FemaleFirstNames ORDER BY firstName")
dbTest("SQL-L2-count", 10, resultsDF.count())

results = [ f"{r[0]}, {r[1]}" for r in resultsDF.collect()]
dbTest("SQL-L2-names-0", "Alesha, 1368", results[0])
dbTest("SQL-L2-names-1", "Alice, 1384", results[1])
dbTest("SQL-L2-names-2", "Bridgette, 1373", results[2])
dbTest("SQL-L2-names-3", "Cristen, 1375", results[3])
dbTest("SQL-L2-names-4", "Jacquelyn, 1381", results[4])
dbTest("SQL-L2-names-5", "Katherin, 1373", results[5])
dbTest("SQL-L2-names-6", "Lashell, 1387", results[6])
dbTest("SQL-L2-names-7", "Louie, 1382", results[7])
dbTest("SQL-L2-names-8", "Lucille, 1384", results[8])
dbTest("SQL-L2-names-9", "Sharyn, 1394", results[9])

print("Tests passed!")

Tests passed!

### Step 2

Display the contents of the temporary view.

In [39]:
%sql
SELECT * FROM Top10FemaleFirstNames


firstName,total
Sharyn,1394
Lashell,1387
Lucille,1384
Alice,1384
Louie,1382
Jacquelyn,1381
Cristen,1375
Bridgette,1373
Katherin,1373
Thresa,1368


## Summary
* Spark SQL queries tables that are backed by physical files
* You can visualize the results of your queries with built-in Databricks graphs

## Review Questions
**Q:** What is the prefix used in databricks cells to execute SQL queries?  
**A:** `%sql`

**Q:** How do temporary views differ from tables?  
**A:** Tables are visible to all users, can be accessed from any notebook, and persist across server resets.  Temporary views are only visible to the current user, in the current notebook, and are gone once the spark session ends.

**Q:** What is the SQL syntax to create a temporary view?  
**A:** ```CREATE OR REPLACE TEMPORARY VIEW <<ViewName>> AS <<Query>>```

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [43]:
%run "./Includes/Classroom-Cleanup"

## Next Steps

Start the next lesson, [Aggregations, JOINs and Nested Queries]($./SSQL 03 - Joins Aggregations ).

## Additional Topics & Resources

* <a href="https://docs.databricks.com/spark/latest/spark-sql/index.html" target="_blank">Spark SQL Reference</a>
* <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html" target="_blank">Spark SQL, DataFrames and Datasets Guide</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>